In [7]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'  # carpeta donde se encuentran los datasets
else:
    FUENTES_DIR = '../Fuentes/' # carpeta LOCAL donde se encuentran los scripts
    DATOS_DIR   = '../Datos/' # carpeta LOCAL donde se encuentran los datasets

import sys
sys.path.append(FUENTES_DIR)

import pandas as pd
import numpy as np
import chardet
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Input, LeakyReLU
from sklearn.model_selection import train_test_split


def openFile(nomArch, sep=None):
    file = DATOS_DIR + nomArch
    #-- detectando la codificación de caracteres usada ----
    with open(file, 'rb') as f:
        result = chardet.detect(f.read()) 
    return pd.read_csv(file, encoding=result['encoding'], sep=sep, engine='python') # or readline if the file is large

2026-01-12 10:20:10.050172: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-12 10:20:10.059973: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-12 10:20:10.389963: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-12 10:20:11.847197: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

#### Ejercicio 7

In [16]:
df =  openFile("Balance.csv")

X = df.drop(columns=["Balance"]).to_numpy()
y = df["Balance"].to_numpy()

binariazer = preprocessing.LabelBinarizer()
y = binariazer.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Normalizar datos
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
# cantidad de pasadas de los datos
EPOCAS = 1000
# cantidad de datos a procesar para actualizar pesos
TAM_LOTE = X.shape[0] // 20

ENTRADAS = X.shape[1]
print("ENTRADAS:", ENTRADAS)
SALIDAS = len(df["Balance"].unique())

ACTIVACION = LeakyReLU()
# ACTIVACION = 'ReLU'
# ACTIVACION = 'tanh'
# ACTIVACION = 'sigmoid'

# OPTIMIZADOR = 'sgd'
OPTIMIZADOR = 'rmsprop'
# OPTIMIZADOR = 'adam'
PACIENCIA = 100

model = Sequential()
model.add(Input(shape=(ENTRADAS,)))
model.add(Dense(20, activation=ACTIVACION))
model.add(Dense(20, activation="tanh"))
model.add(Dense(20, activation="tanh"))
model.add(Dense(SALIDAS, activation='softmax'))
# model.add(Dense(SALIDAS))

model.summary()

# obtiene la arquitectura para el modelo y lo compila
model.compile(optimizer=OPTIMIZADOR, loss='categorical_crossentropy', metrics = ['accuracy'])

ENTRADAS: 4


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_32 (Dense)                │ (None, 20)             │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 3)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,003 (3.92 KB)

 Trainable params: 1,003 (3.92 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# El parámetro patience indica la cantidad de epocas que deben transcurrir
# sin mejoras en el entrenamiento
early_stop = tf.keras.callbacks.EarlyStopping(
                                    monitor='val_loss',
                                    patience=PACIENCIA,
                                    restore_best_weights=True
                                )

# Dividir datos de entrenamiento en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Entrena el modelo y guarda la historia del progreso
history = model.fit(
                x = X_train,
                y = y_train,
                batch_size = TAM_LOTE,
                epochs = EPOCAS,
                validation_data = (X_val, y_val),
                callbacks=[early_stop],
            )

print('\n'+'-'*80)
print('Epocas utilizadas: %d' % len(history.epoch))

# %% Evalua e informa resultado de entrenamiento, validación y testeo
# evalua el modelo con los datos de entreanmiento
pred = model.evaluate(X_train, y_train, verbose=False)
print('\nEfectividad del modelo con datos de entrenamiento para:' )
print(" - Accuracy: %6.2f%%" % (pred[1]*100))
print(" - Pérdida : %9.5f" % (pred[0]))

# evalua el modelo con los datos de validacion
pred = model.evaluate(X_val, y_val, verbose=False)
print('\nEfectividad del modelo con datos de validacion para:' )
print(" - Accuracy: %6.2f%%" % (pred[1]*100))
print(" - Pérdida : %9.5f" % (pred[0]))


Epoch 1/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6500 - loss: 0.8486 - val_accuracy: 0.8400 - val_loss: 0.6419
Epoch 2/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8250 - loss: 0.6304 - val_accuracy: 0.8700 - val_loss: 0.5161
Epoch 3/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8700 - loss: 0.5185 - val_accuracy: 0.8800 - val_loss: 0.4395
Epoch 4/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8900 - loss: 0.4411 - val_accuracy: 0.8900 - val_loss: 0.3868
Epoch 5/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8950 - loss: 0.3863 - val_accuracy: 0.8900 - val_loss: 0.3508
Epoch 6/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8975 - loss: 0.3477 - val_accuracy: 0.8900 - val_loss: 0.3245
Epoch 7/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8975 - loss: 0.3172 - val_accuracy: 0.9000 - val_loss: 0.3032
Epoch 8/1000
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9000 - loss: 0.2952 - val_accuracy: 0

KeyboardInterrupt: 

In [ ]:
y_pred = model.evaluate(X_test, y_test, verbose=False)
print('\nEfectividad del modelo con datos de testeo para:' )
print(" - Accuracy: %6.2f%%" % (y_pred[1]*100))
print(" - Pérdida : %9.5f" % (y_pred[0]))


Efectividad del modelo con datos de testeo para:
 - Accuracy:  96.80%
 - Pérdida :   0.05529


#### Ejercicio 8

In [ ]:
df = openFile('Iris.csv')

X = df.drop(columns=["class"]).to_numpy()
y = df["class"].to_numpy()

# -----------  Normalizar datos  -----------
y_encoder = preprocessing.LabelEncoder()
y = y_encoder.fit_transform(y)
# print(y)

y_norm = tf.keras.utils.to_categorical(y)
# print(y_binariazer)

X_train, X_test, y_train, y_test = train_test_split(X, y_norm, test_size=0.2, random_state=42, shuffle=True)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# cantidad de pasadas de los datos
EPOCAS = 1000

# cantidad de datos a procesar para actualizar pesos
TAM_LOTE = X.shape[0] // 20

ENTRADAS = X.shape[1]
SALIDAS = len(df["class"].unique())

ACTIVACION = LeakyReLU()
# ACTIVACION = 'ReLU'
# ACTIVACION = 'tanh'
# ACTIVACION = 'sigmoid'

# OPTIMIZADOR = 'sgd'
OPTIMIZADOR = 'rmsprop'
# OPTIMIZADOR = 'adam'
PACIENCIA = 100

model = Sequential()
model.add(Input(shape=(ENTRADAS,)))
model.add(Dense(20, activation=ACTIVACION))
model.add(Dense(10, activation="tanh"))
model.add(Dense(5, activation="tanh"))
model.add(Dense(SALIDAS, activation='softmax'))
# model.add(Dense(SALIDAS))

model.summary()

# obtiene la arquitectura para el modelo y lo compila
model.compile(optimizer=OPTIMIZADOR, loss='categorical_crossentropy', metrics = ['accuracy'])

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 20)             │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 5)              │            55 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 3)              │            18 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 383 (1.50 KB)

 Trainable params: 383 (1.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

# Entrena el modelo y guarda la historia del progreso
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=PACIENCIA,
    restore_best_weights=True
)

# Entrena el modelo y guarda la historia del progreso
history = model.fit(X_train, y_train, validation_data=(
    X_val, y_val), epochs=EPOCAS, batch_size=TAM_LOTE, callbacks=[early_stop])

print('\n'+'-'*80)
print('Epocas utilizadas: %d' % len(history.epoch))

# %% Evalua e informa resultado de entrenamiento, validación y testeo
# evalua el modelo con los datos de entreanmiento
train_pred = model.evaluate(X_train, y_train, verbose=False)
print('\nEfectividad del modelo con datos de entrenamiento para:' )
print(" - Accuracy: %6.2f%%" % (train_pred[1]*100))
print(" - Pérdida : %9.5f" % (train_pred[0]))

# evalua el modelo con los datos de validacion
val_pred = model.evaluate(X_val, y_val, verbose=False)
print('\nEfectividad del modelo con datos de validacion para:' )
print(" - Accuracy: %6.2f%%" % (val_pred[1]*100))
print(" - Pérdida : %9.5f" % (val_pred[0]))


Epoch 1/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.3125 - loss: 1.1862 - val_accuracy: 0.1667 - val_loss: 1.3409
Epoch 2/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3854 - loss: 1.0590 - val_accuracy: 0.2083 - val_loss: 1.2364
Epoch 3/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4792 - loss: 0.9619 - val_accuracy: 0.3333 - val_loss: 1.1429
Epoch 4/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5938 - loss: 0.8757 - val_accuracy: 0.3333 - val_loss: 1.0596
Epoch 5/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6458 - loss: 0.7971 - val_accuracy: 0.4167 - val_loss: 0.9811
Epoch 6/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6771 - loss: 0.7296 - val_accuracy: 0.5000 - val_loss: 0.9155
Epoch 7/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7292 - loss: 0.6708 - val_accuracy: 0.6250 - val_loss: 0.8561
Epoch 8/1000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7500 - loss: 0.6179 - val_accuracy: 0

In [ ]:
y_pred = model.evaluate(X_test, y_test, verbose=False)
print('\nEfectividad del modelo con datos de testeo para:' )
print(" - Accuracy: %6.2f%%" % (y_pred[1]*100))
print(" - Pérdida : %9.5f" % (y_pred[0]))


Efectividad del modelo con datos de testeo para:
 - Accuracy:  96.67%
 - Pérdida :   0.08761


In [ ]:
# Cálculo manual de métricas
# Obtener las probabilidades predichas por el modelo
y_test_pred_probs = model.predict(X_test)
y_train_pred_probs = model.predict(X_train)

# Convertir las probabilidades a índices de clase
y_test_pred_indices = np.argmax(y_test_pred_probs, axis=1)
y_train_pred_indices = np.argmax(y_train_pred_probs, axis=1)

# Transformar los índices a etiquetas originales
y_test_pred_transformado = y_encoder.inverse_transform(y_test_pred_indices)
y_train_pred_transformado = y_encoder.inverse_transform(y_train_pred_indices)
print(y_test_pred_transformado)

# Si quieres calcular la efectividad manualmente:
y_test_true_indices = np.argmax(y_test, axis=1)
y_train_true_indices = np.argmax(y_train, axis=1)
y_test_true_transformado = y_encoder.inverse_transform(y_test_true_indices)
y_train_true_transformado = y_encoder.inverse_transform(y_train_true_indices)

print('Efectividad test: %6.2f%%' % (100 * (y_test_pred_transformado == y_test_true_transformado).sum() / len(y_test_true_transformado)))
print('Efectividad train: %6.2f%%' % (100 * (y_train_pred_transformado == y_train_true_transformado).sum() / len(y_train_true_transformado)))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
['Iris-versicolor' 'Iris-setosa' 'Iris-virginica' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor' 'Iris-virginica'
 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-versicolor' 'Iris-versicolor' 'Iris-virginica'
 'Iris-setosa' 'Iris-virginica' 'Iris-setosa' 'Iris-virginica'
 'Iris-virginica' 'Iris-virginica' 'Iris-virginica' 'Iris-virginica'
 'Iris-setosa' 'Iris-setosa']
Efectividad test:  96.67%
Efectividad train:  97.92%


In [ ]:
def calcular_metricas(conf_mat):
    precision = np.zeros(conf_mat.shape[0])
    for i in range(0, len(conf_mat)):
        precision[i] = conf_mat[i][i]/sum(conf_mat.T[i])

    recall = np.zeros(conf_mat.shape[0])
    for i in range(0, len(conf_mat)):
        recall[i] = conf_mat[i][i]/sum(conf_mat[i])

    f1_score = 2* (precision*recall) /(precision+recall)

    accuracy =  0
    for i in range(0, len(conf_mat)):
        accuracy+=conf_mat[i][i]
    accuracy/= conf_mat.sum()

    return ( precision, recall, f1_score, accuracy )

# el parámetro metricas es una tupla ( precision, recall, f1_score, accuracy )
def imprimir_metricas( metricas ):
    (precision, recall, f1_score, accuracy) = metricas
    print('\n clase   precision    recall    f1-score')
    for i in range(0, len(precision)):
        print('%5d %10.2f %10.2f %10.2f' % (i, precision[i], recall[i], f1_score[i]))
    print('\naccuracy: %6.2f\n' % accuracy)

In [ ]:
conf_mat_test = tf.math.confusion_matrix(y_test_true_indices, y_test_pred_indices).numpy()

metricas = calcular_metricas(conf_mat_test)
imprimir_metricas(metricas)



 clase   precision    recall    f1-score
    0       1.00       1.00       1.00
    1       1.00       0.89       0.94
    2       0.92       1.00       0.96

accuracy:   0.97



In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_true_transformado, y_test_pred_transformado, zero_division=0))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      0.89      0.94         9
 Iris-virginica       0.92      1.00      0.96        11

       accuracy                           0.97        30
      macro avg       0.97      0.96      0.97        30
   weighted avg       0.97      0.97      0.97        30



#### Ejercicio 9

In [19]:
df = openFile('autos.csv')

# Elimina filas con valores faltantes en la columna 'price'
df = df[df['price'] != '?']

df.replace('?', np.nan, inplace=True)

#-- seleccionar los atributos numéricos --
df["normalized-losses"] = pd.to_numeric(df["normalized-losses"], errors='coerce')
df["bore"] = pd.to_numeric(df["bore"], errors='coerce')
df["stroke"] = pd.to_numeric(df["stroke"], errors='coerce')
df["horsepower"] = pd.to_numeric(df["horsepower"], errors='coerce')
df["peak-rpm"] = pd.to_numeric(df["peak-rpm"], errors='coerce')
df["price"] = pd.to_numeric(df["price"], errors='coerce')

df = df.select_dtypes(include = ["int16", "int32", "int64", "float16", "float32", "float64"])

values = {
    'normalized-losses': df['normalized-losses'].mean(),
          'bore': df['bore'].mean(),
            'stroke': df['stroke'].mean(),
            'horsepower': df['horsepower'].mean(),
            'peak-rpm': df['peak-rpm'].mean(),
}

df = df.fillna(value=values)


Predicción del atributo "price"

In [20]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

X = df.drop(columns=["price"]).to_numpy()
y = df["price"].to_numpy().reshape(-1, 1) # para que y sea una matriz columna

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Normalizar datos
scaler_x = preprocessing.StandardScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

scaler_y = preprocessing.StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Parámetros de la red
EPOCAS = 1000
TAM_LOTE = 50
PACIENCIA = 15
ENTRADAS = X.shape[1]

# Definir combinaciones a probar
optimizadores = ['SGD', 'RMSprop', 'Adam']
activaciones = ['tanh', 'sigmoid', 'ReLU', 'LeakyReLU']

# Crear DataFrame para resultados
resultados = []

# Experimento sistemático
for opt_name in optimizadores:
    for act_name in activaciones:
        print(f"\n🔄 Probando: {opt_name} + {act_name}")

        total_epochs = 0
        total_ecm = 0
        for i in range(20):  # Repetir cada combinación 20 veces
            # Crear modelo
            model = Sequential()
            model.add(Input(shape=(ENTRADAS,)))
            
            # Seleccionar función de activación
            if act_name == 'tanh':
                activacion = 'tanh'
            elif act_name == 'sigmoid':
                activacion = 'sigmoid'
            elif act_name == 'ReLU':
                activacion = 'relu'
            elif act_name == 'LeakyReLU':
                activacion = LeakyReLU()
            
            model.add(Dense(10, activation=activacion))
            model.add(Dense(1, activation='linear'))  # Regresión
            
            # Seleccionar optimizador
            if opt_name == 'SGD':
                optimizador = SGD(momentum=0.9)
            elif opt_name == 'RMSprop':
                optimizador = RMSprop()
            elif opt_name == 'Adam':
                optimizador = Adam()
            
            # Compilar modelo
            model.compile(optimizer=optimizador, loss='mse', metrics=['mae'])
            
            # Early stopping
            early_stop = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=PACIENCIA,
                restore_best_weights=True
            )
            
            # Dividir train en train/validation
            X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
                X_train_scaled, y_train_scaled, test_size=0.2, shuffle=True
            )
            
            # Entrenar modelo
            history = model.fit(
                X_train_split, y_train_split,
                epochs=EPOCAS,
                batch_size=TAM_LOTE,
                validation_data=(X_val_split, y_val_split),
                callbacks=[early_stop],
                verbose=0
            )
            
            # Calcular métricas
            epocas_usadas = len(history.history['loss'])
            total_epochs += epocas_usadas
            
            # Evaluar en test
            test_loss, test_mae = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
            
            # Hacer predicciones para calcular ECM en escala original
            y_pred_scaled = model.predict(X_test_scaled, verbose=0)
            y_pred_real = scaler_y.inverse_transform(y_pred_scaled)
            y_test_real = scaler_y.inverse_transform(y_test_scaled)

            # Calcular ECM en escala original
            ecm = np.mean((y_test_real - y_pred_real)**2)
            total_ecm += ecm
        
        # ECM en escala original
        ecm_promedio = total_ecm / 20
        epocas_promedio = total_epochs // 20
        
        # Guardar resultados
        resultado = {
            'Optimizador': opt_name,
            'Función activación': act_name,
            'Épocas promedio': epocas_promedio,
            'ECM Promedio': ecm_promedio
        }
        resultados.append(resultado)
        
        print(f"   ✅ Épocas promedio: {epocas_promedio}, ECM: {ecm_promedio:,.0f}")
            
# Crear DataFrame con resultados
df_resultados = pd.DataFrame(resultados)

# Crear tabla pivote
tabla_epocas = df_resultados.pivot(index='Función activación', 
                                   columns='Optimizador', 
                                   values='Épocas promedio')

tabla_ecm = df_resultados.pivot(index='Función activación', 
                                columns='Optimizador', 
                                values='ECM Promedio')

print("\n" + "="*80)
print("RESULTADOS FINALES")
print("="*80)

print("\n📊 ÉPOCAS PROMEDIO:")
print(tabla_epocas)

print("\n📊 ECM PROMEDIO:")
print(tabla_ecm)

# Encontrar mejor combinación
df_validos = df_resultados[df_resultados['ECM Promedio'] != 'Error'].copy()
df_validos['ECM Promedio'] = pd.to_numeric(df_validos['ECM Promedio'])

# ✅ Agregar tabla de RMSE (más interpretable)
df_resultados['RMSE'] = np.sqrt(df_resultados['ECM Promedio'])
tabla_rmse = df_resultados.pivot(
    index='Función activación', 
    columns='Optimizador', 
    values='RMSE'
)

print("\n📊 RMSE PROMEDIO (Error en dólares):")
print(tabla_rmse.applymap(lambda x: f"${x:,.0f}"))

# Encontrar mejor combinación
mejor = df_resultados.loc[df_resultados['ECM Promedio'].idxmin()]
print(f"\n🏆 MEJOR COMBINACIÓN:")
print(f"   Optimizador: {mejor['Optimizador']}")
print(f"   Activación: {mejor['Función activación']}")
print(f"   Épocas: {mejor['Épocas promedio']:.1f}")
print(f"   ECM: {mejor['ECM Promedio']:,.0f}")
print(f"   RMSE: ${mejor['RMSE']:,.0f}")

# Tabla completa
print(f"\n📋 TABLA COMPLETA DE RESULTADOS:")
df_display = df_resultados.copy()
df_display['ECM Promedio'] = df_display['ECM Promedio'].apply(lambda x: f"{x:,.0f}")
df_display['RMSE'] = df_display['RMSE'].apply(lambda x: f"${x:,.0f}")
df_display['Épocas promedio'] = df_display['Épocas promedio'].round(1)
print(df_display.to_string(index=False))


🔄 Probando: SGD + tanh
   ✅ Épocas promedio: 71, ECM: 35,361,175

🔄 Probando: SGD + sigmoid
   ✅ Épocas promedio: 139, ECM: 30,104,716

🔄 Probando: SGD + ReLU
   ✅ Épocas promedio: 66, ECM: 22,953,349

🔄 Probando: SGD + LeakyReLU
   ✅ Épocas promedio: 42, ECM: 28,308,042

🔄 Probando: RMSprop + tanh
   ✅ Épocas promedio: 188, ECM: 30,886,861

🔄 Probando: RMSprop + sigmoid
   ✅ Épocas promedio: 334, ECM: 33,150,822

🔄 Probando: RMSprop + ReLU
   ✅ Épocas promedio: 184, ECM: 24,067,889

🔄 Probando: RMSprop + LeakyReLU
   ✅ Épocas promedio: 164, ECM: 21,494,081

🔄 Probando: Adam + tanh
   ✅ Épocas promedio: 332, ECM: 27,460,848

🔄 Probando: Adam + sigmoid
   ✅ Épocas promedio: 477, ECM: 32,151,434

🔄 Probando: Adam + ReLU
   ✅ Épocas promedio: 237, ECM: 29,298,616

🔄 Probando: Adam + LeakyReLU
   ✅ Épocas promedio: 211, ECM: 26,306,921

RESULTADOS FINALES

📊 ÉPOCAS PROMEDIO:
Optimizador         Adam  RMSprop  SGD
Función activación                    
LeakyReLU            211      164   4

/tmp/ipykernel_29108/1816075736.py:154: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(tabla_rmse.applymap(lambda x: f"${x:,.0f}"))


#### Ejercicio 10

In [2]:
import os
import pandas as pd
import numpy as np
from skimage import io
from skimage.measure import regionprops, label
from skimage.filters import threshold_otsu
from skimage.morphology import closing, disk
from skimage.segmentation import clear_border

In [3]:

img_path = "/home/santi/Documentos/Cuarto/Deep Learning/Prácticas/Datos/archive/fingers/test/0a4d7cbc-2522-4e51-968a-1a86d3b7ee19_5L.png"

imagen = io.imread(img_path)

# busca umbral global con método estadístico de Otsu
umbral = threshold_otsu(imagen)

# binariza la imagen
imagen_bn = (imagen > umbral)*1

# cierra pequeños huecos/cortes que pudiera tener la imagen de la mano
imagen_bn = closing(imagen_bn, disk(2))

# remueve artefactos que pudiera tener la imagen en los bordes
imagen_lista = clear_border(imagen_bn)

# obtiene valores geométricos a partir de las regiones (objetos "aislados") en la imagen
regiones = regionprops(imagen_lista)

# datos de la primera región. Debería ser la única si la mano fue segmentada correctamente
region = regiones[0]

for prop in region:
  #if prop not in ['convex_image', 'coords', 'image_filled', 'image', 'image_convex', 'moments', 'moments_central', 'moments_normalized', 'moments_hu']:
  print('%20s:    '% prop, region[prop])

                area:     1892.0
           area_bbox:     4278.0
         area_convex:     3064.0
         area_filled:     1892.0
   axis_major_length:     67.49455955508728
   axis_minor_length:     47.37266683356695
                bbox:     (24, 29, 93, 91)
            centroid:     (np.float64(63.58562367864693), np.float64(62.811310782241016))
      centroid_local:     [39.58562368 33.81131078]
              coords:     [[24 56]
 [24 57]
 [24 58]
 ...
 [92 57]
 [92 58]
 [92 59]]
        eccentricity:     0.7123012628261987
equivalent_diameter_area:     49.08125119267976
        euler_number:     1
              extent:     0.442262739597943
  feret_diameter_max:     70.2353187506115
               image:     [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
        ima

In [13]:
def extract_features_from_image(img_path):
    """
    Extrae características de una imagen usando regionprops
    """
    try:
        # Leer imagen
        img = io.imread(img_path)
        
        # Si la imagen no está binarizada, binarizarla
        if img.max() > 1:
            # Binarizar usando Otsu
            threshold = threshold_otsu(img)
            img_binary = img > threshold
        else:
            img_binary = img.astype(bool)
        
        # ✅ CORREGIDO: Usar disk en lugar de square
        img_clean = closing(img_binary, disk(2))
        img_clean = clear_border(img_clean)
        
        # Etiquetar regiones conectadas
        labeled_img = label(img_clean, connectivity=2)
        
        # Obtener propiedades de las regiones
        regions = regionprops(labeled_img)
        
        if len(regions) == 0:
            return None
        
        # Tomar la región más grande (probablemente la mano)
        largest_region = max(regions, key=lambda r: r.area)
        
        # Extraer características específicas
        features = {
            'filename': os.path.basename(img_path),
            'filled_area': largest_region.filled_area,
            'major_axis_length': largest_region.major_axis_length,
            'minor_axis_length': largest_region.minor_axis_length,
            'perimeter': largest_region.perimeter,
            'eccentricity': largest_region.eccentricity,
            'solidity': largest_region.solidity,
            'extent': largest_region.extent,
            'num_of_fingers': img_path.split('_')[-1].split('.')[0][0] # Extrae número de dedos del nombre del archivo
        }
        
        return features
        
    except Exception as e:
        print(f"Error procesando {img_path}: {e}")
        return None


In [14]:
def process_folder(folder_path, output_filename):
    """
    Procesa todas las imágenes en una carpeta y guarda las características en CSV
    """
    print(f"Procesando carpeta: {folder_path}")
    
    # Lista para almacenar todas las características
    all_features = []
    
    # Obtener lista de archivos de imagen
    image_extensions = ['.png', '.jpg', '.jpeg', '.bmp', '.tiff']
    image_files = []
    
    for filename in os.listdir(folder_path):
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            image_files.append(filename)
    
    print(f"Encontradas {len(image_files)} imágenes")
    
    # Procesar cada imagen
    for i, img_name in enumerate(image_files):
        img_path = os.path.join(folder_path, img_name)
        
        # Mostrar progreso
        if (i + 1) % 50 == 0 or (i + 1) == len(image_files):
            print(f"Procesando imagen {i + 1}/{len(image_files)}: {img_name}")
        
        # Extraer características
        features = extract_features_from_image(img_path)
    
        if features is not None:
            all_features.append(features)
    
    # Crear DataFrame
    df = pd.DataFrame(all_features)
    
    # Guardar CSV
    df.to_csv(output_filename, index=False)
    print(f"Archivo guardado: {output_filename}")
    print(f"Características extraídas de {len(df)} imágenes")
    
    return df

In [15]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module="skimage.morphology")


# Rutas de las carpetas
base_path = "/home/santi/Documentos/Cuarto/Deep Learning/Prácticas/Datos/archive/fingers/"
train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")

# Verificar que las carpetas existen
if not os.path.exists(train_path):
    print(f"❌ No se encuentra la carpeta: {train_path}")
else:
    print(f"✅ Carpeta train encontrada: {len(os.listdir(train_path))} archivos")

if not os.path.exists(test_path):
    print(f"❌ No se encuentra la carpeta: {test_path}")
else:
    print(f"✅ Carpeta test encontrada: {len(os.listdir(test_path))} archivos")

# Procesar carpeta train
print("\n" + "="*50)
print("PROCESANDO CARPETA TRAIN")
print("="*50)
train_df = process_folder(train_path, "train_fingers_features.csv")

# Procesar carpeta test
print("\n" + "="*50)
print("PROCESANDO CARPETA TEST")
print("="*50)
test_df = process_folder(test_path, "test_fingers_features.csv")

# Mostrar resumen de los datos
print("\n" + "="*50)
print("RESUMEN DE DATOS EXTRAÍDOS")
print("="*50)

print("\nDATOS DE ENTRENAMIENTO:")
print(f"Número de imágenes procesadas: {len(train_df)}")
print("\nPrimeras 5 filas:")
print(train_df.head())
print("\nEstadísticas:")
print(train_df.describe())

print("\nDATOS DE PRUEBA:")
print(f"Número de imágenes procesadas: {len(test_df)}")
print("\nPrimeras 5 filas:")
print(test_df.head())
print("\nEstadísticas:")
print(test_df.describe())

# Verificar si hay valores faltantes
print("\n" + "="*30)
print("VERIFICACIÓN DE CALIDAD")
print("="*30)

print("\nValores faltantes en train:")
print(train_df.isnull().sum())

print("\nValores faltantes en test:")
print(test_df.isnull().sum())

# Mostrar rangos de las características
print("\nRangos de características (train):")
numeric_cols = ['filled_area', 'major_axis_length', 'minor_axis_length', 
                'perimeter', 'eccentricity', 'solidity', 'extent']

for col in numeric_cols:
    if col in train_df.columns:
        print(f"{col:20s}: {train_df[col].min():8.2f} - {train_df[col].max():8.2f}")


✅ Carpeta train encontrada: 18000 archivos
✅ Carpeta test encontrada: 3600 archivos

PROCESANDO CARPETA TRAIN
Procesando carpeta: /home/santi/Documentos/Cuarto/Deep Learning/Prácticas/Datos/archive/fingers/train
Encontradas 18000 imágenes
Procesando imagen 50/18000: 33fc7ff2-bf5c-44a4-9dd8-a5bb502b52ce_3L.png
Procesando imagen 100/18000: b1406cbe-82b8-43e7-a4ad-d7b14635ad84_2R.png
Procesando imagen 150/18000: 338d8f16-735f-4ef7-9be8-b75f7fbc7763_4L.png
Procesando imagen 200/18000: dbb9bdab-f31e-4e6c-9f53-d7bac457802a_3L.png
Procesando imagen 250/18000: 43528d0b-62ad-4ba3-b1a9-6a6bee4cccbe_0R.png
Procesando imagen 300/18000: d596a2f5-526e-4a1d-ac8b-35c1f7ddba47_4L.png
Procesando imagen 350/18000: bb483295-9c42-431e-9d00-f6f8c25c1b0f_3L.png
Procesando imagen 400/18000: a9768946-1151-4ac5-8ee3-4d0aed510d21_5L.png
Procesando imagen 450/18000: 7f7c10e0-420c-43f0-8978-771b8093c93b_4L.png
Procesando imagen 500/18000: 03c72043-b23f-48bc-88e4-e6de323b7aff_4R.png
Procesando imagen 550/18000: 3cc

In [35]:
from tensorflow.keras.layers import Dropout

X_train = train_df.drop(columns=["filename", "num_of_fingers"]).to_numpy()
y_train_raw = train_df["num_of_fingers"].astype(int).to_numpy()
y_train = to_categorical(y_train_raw, num_classes=6)

X_test = test_df.drop(columns=["filename", "num_of_fingers"]).to_numpy()
y_test_raw = test_df["num_of_fingers"].astype(int).to_numpy()
y_test = to_categorical(y_test_raw, num_classes=6)

entradas = X_train.shape[1]
print("ENTRADAS TRAIN:", entradas)
salidas = 6
print("SALIDAS TRAIN:", salidas)

model = Sequential()
model.add(Input(shape=(entradas,)))
model.add(Dense(20, activation="linear"))
model.add(Dense(20, activation=LeakyReLU()))
model.add(Dense(20, activation=LeakyReLU()))
model.add(Dropout(0.2))
model.add(Dense(20, activation=LeakyReLU()))
model.add(Dense(20, activation=LeakyReLU()))
model.add(Dropout(0.2))
model.add(Dense(salidas, activation='softmax'))

model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy', 'mse'])

early_stop = tf.keras.callbacks.EarlyStopping(
                                    monitor='val_loss',
                                    patience=50,
                                    restore_best_weights=True
                                )

history = model.fit(
                x = X_train,
                y = y_train,
                batch_size = 110,
                epochs = 700,
                validation_split = 0.2,
                callbacks=[early_stop],
            )

# Evaluar el modelo con los datos de testeo
y_pred = model.evaluate(X_test, y_test, verbose=False)

print('\nEfectividad del modelo con datos de testeo para:' )
print(" - Accuracy: %6.2f%%" % (y_pred[1]*100))
print(" - Pérdida : %9.5f" % (y_pred[0]))

ENTRADAS TRAIN: 7
SALIDAS TRAIN: 6


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_30 (Dense)                │ (None, 20)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 6)              │           126 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,966 (7.68 KB)

 Trainable params: 1,966 (7.68 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/700
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1831 - loss: 51.8536 - mse: 0.2432 - val_accuracy: 0.1842 - val_loss: 1.9394 - val_mse: 0.1535
Epoch 2/700
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2525 - loss: 2.0709 - mse: 0.1518 - val_accuracy: 0.4419 - val_loss: 1.3890 - val_mse: 0.1161
Epoch 3/700
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3235 - loss: 1.6426 - mse: 0.1327 - val_accuracy: 0.5089 - val_loss: 1.2847 - val_mse: 0.1092
Epoch 4/700
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3808 - loss: 1.4858 - mse: 0.1230 - val_accuracy: 0.6278 - val_loss: 1.1301 - val_mse: 0.0987
Epoch 5/700
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4136 - loss: 1.3879 - mse: 0.1173 - val_accuracy: 0.5694 - val_loss: 1.0613 - val_mse: 0.0946
Epoch 6/700
131/131 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4873 - loss: 1.2210 - mse: 0.1054 - val_accuracy: 0.7314 - val_loss: 0.8761 - val_mse: 0.0770
Epoch 7/700
131/131 ━━━━━━━━━━━━━